In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126221")
exp = Experiment(workspace=ws, name="quick-starts-ws-126221")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126221
Azure region: southcentralus
Subscription id: 5e9e5b82-0802-460e-849e-48901a64399b
Resource group: aml-quickstarts-126221


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = 'uda-ts-cluster'
try:
    cpu_cluster = ComputeTarget(ws, cpu_cluster_name)
    print("Cluster Exists")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, idle_seconds_before_scaledown=240)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    print("Creating Cluster")
 
cpu_cluster.wait_for_completion(show_output=True)

Creating Cluster
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.1, 100),
        "--max_iter": choice(10, 50, 100, 250, 500, 750, 1000)
    })

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', 
            compute_target=cpu_cluster,
            vm_size='STANDARD_D2_V2', 
            vm_priority='lowpriority',
            entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=30,
                             max_concurrent_runs=4)

In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
import joblib
# Get your best run and save the model from that run.
hyperdrive_run.wait_for_completion(show_output=True)

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best run: ', best_run.id, 'metrics: ', best_run_metrics,'parameters: ', parameter_values)

#best_run.register_model(model_name='best_hyper_model', model_path='outputs/automl_model.joblib')

RunId: HD_bdf5d4c2-3d87-4fb4-9413-c1ce635d5480
Web View: https://ml.azure.com/experiments/quick-starts-ws-126221/runs/HD_bdf5d4c2-3d87-4fb4-9413-c1ce635d5480?wsid=/subscriptions/5e9e5b82-0802-460e-849e-48901a64399b/resourcegroups/aml-quickstarts-126221/workspaces/quick-starts-ws-126221

Execution Summary
RunId: HD_bdf5d4c2-3d87-4fb4-9413-c1ce635d5480
Web View: https://ml.azure.com/experiments/quick-starts-ws-126221/runs/HD_bdf5d4c2-3d87-4fb4-9413-c1ce635d5480?wsid=/subscriptions/5e9e5b82-0802-460e-849e-48901a64399b/resourcegroups/aml-quickstarts-126221/workspaces/quick-starts-ws-126221

Best run:  HD_bdf5d4c2-3d87-4fb4-9413-c1ce635d5480_0 metrics:  {'Regularization Strength:': 2.1796423794543447, 'Max iterations:': 250, 'Accuracy': 0.910152657715652} parameters:  ['--C', '2.1796423794543447', '--max_iter', '250']


In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datapath = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=datapath)

In [19]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

import pandas
training_data = pandas.concat([x_train, y_train], axis=1)

default_datastore = ws.get_default_datastore()
TabularDatasetFactory.register_pandas_dataframe(training_data, default_datastore, name='automl_training_data')
from azureml.core import Dataset
dataset = Dataset.get_by_name(ws, name='automl_training_data')

WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/26c9eee1-521a-454f-828c-6b6de4e169fd/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [21]:
# Submit your automl run

autoExp = Experiment(workspace=ws, name="udacity-automl")
automl_run = autoExp.submit(automl_config,show_output=True)
RunDetails(automl_run).show()

Running on remote.
Running on remote compute: uda-ts-cluster
Parent Run ID: AutoML_de02d4c4-206d-43c0-97ba-fe91a85291f8

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of t

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [23]:
# Retrieve and save your best automl model.

automl_run.wait_for_completion()
best_automl_run, best_automl_model = automl_run.get_output()
best_automl_run
#best_automl_run.register_model(model_name='best_automl_model')

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-automl,AutoML_de02d4c4-206d-43c0-97ba-fe91a85291f8_27,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [24]:
cpu_cluster.delete()